# General Notebook Configuration

In [ ]:
# LoRA installation
%%capture
! pip install --no-deps peft bitsandbytes
! pip install -U trl==0.9.4

# Import LoRA Packages
from peft import LoraConfig, TaskType, get_peft_model
from trl import SFTTrainer
from transformers import BitsAndBytesConfig

In [ ]:
# Import General Packages
from tqdm import tqdm
from datasets import Dataset
from typing import Union
from transformers import (
    NllbTokenizerFast,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    EarlyStoppingCallback,
    DataCollatorForSeq2Seq
)
import json
import io
import random
import numpy as np
import torch
import os
import datasets
import datetime
import gc

In [ ]:
# Import General Packages
from tqdm import tqdm
from datasets import Dataset
from typing import Union
from transformers import NllbTokenizerFast, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, EarlyStoppingCallback
import json
import io
import random
import numpy as np
import torch
import os
import datasets
import datetime
import gc
import re

In [ ]:
# Color messages function
def print_mess (mess:str, color="green") -> None:
  match color:
    case "green":
      print (f"\033[92m{mess}\033[0m")
    case "red":
      print (f"\033[91m{mess}\033[0m")
    case "yellow":
      print (f"\033[93m{mess}\033[0m")
    case "white":
      print (f"\033[97m{mess}\033[0m")

In [ ]:
# Environmente identification
try:
  from google.colab import auth
  from googleapiclient.errors import HttpError
  from googleapiclient.discovery import build
  from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload, MediaFileUpload
  IS_COLAB = True
  print_mess ("Running in Google Colaboratoy Environment.")
except ImportError:
  IS_COLAB = False
  print_mess ("Notebook not running in Gooogle Colaboratory Environment.", "yellow")

Running in Google Colaboratoy Environment.


In [ ]:
# Google connection and authentication
if IS_COLAB:
  auth.authenticate_user ()
  drive_service = build ("drive","v3")

In [ ]:
# Set Seeds for Reproducibility
seed = 3407
random.seed (seed)
np.random.seed (seed)
torch.manual_seed (seed)
if torch.cuda.is_available ():
  torch.cuda.manual_seed_all (seed)

# Dataset' Downloading

In [ ]:
# Main Model Folder
main_model_folder_id = "1hg2OzG02rISe2O0LvmPvwGZP6pvKKu-8"
common_model_folder_name = "Translation_Intent_Model_"

# Evaluation Dataset
main_dataset_folder_id = "1jDVh28jOvtTU6Wsz_ZhlYrUT1plT9KXb"
common_dataset_folder_name = "RMI_Intentions_Dataset_"
dataset_name = "RMI_Intentions_Evaluation_Dataset.json"

# Result Folder
main_model_result_name = "Model_Versions"

# Dataset download directory
dataset_download_dir = "/content/Datasets"
os.makedirs(dataset_download_dir, exist_ok=True)

# Model download directory
model_download_dir = "/content/NLLB-200-Distilled-600M-Trained"
os.makedirs(model_download_dir, exist_ok=True)

In [ ]:
# Identify most recent Dataset Folder function.
def search_folder (main_folder_id:str, initial_folder_name:str) -> Union[None, str]:
  # Search most recent folder
  match_list = drive_service.files ().list (
      q = (f"'{main_folder_id}' in parents "
           f"and mimeType = 'application/vnd.google-apps.folder' "
           f"and name contains '{initial_folder_name}'"),
      spaces = "drive",
      fields = "nextPageToken, files(id, name, createdTime)",
      orderBy = "createdTime desc"
  ).execute ()

  # Obtain most recent Folder Dataset ID
  try:
    return match_list.get ("files", [])[0]["id"]
  except IndexError:
    return None

In [ ]:
# Identify Dataset function
def search_dataset (dataset_name:str, dataset_folder_id:str) -> Union[None, str]:
  # Search most recent Dataset
  match_list = drive_service.files ().list (
      q = f"'{dataset_folder_id}' in parents and name = '{dataset_name}'",
      spaces = "drive",
      fields = "nextPageToken, files(id, name, createdTime)",
      orderBy = "createdTime desc"
  ).execute ()

  # Obtaint most recent Dataset ID
  try:
    return match_list.get ("files", [])[0]["id"]
  except IndexError:
    return None

In [ ]:
# Identify all files in a folder
def search_files (folder_id:str) -> Union [None, list]:
  # Search all files
  match_file_list = drive_service.files ().list (
      q = f"'{folder_id}' in parents",
      spaces = "drive",
      fields = "nextPageToken, files (id, name)"
  ).execute ()

  # Obtain id files
  try:
    return [
        [match_file["id"] for match_file in match_file_list.get("files", [])],
        [match_file["name"] for match_file in match_file_list.get("files", [])]
    ]
  except IndexError:
    return None

In [ ]:
# Download file function
def download_and_save_file (file_id:str, file_name:str, local_path="") -> Union[None, str]:
  # Request to download File
  req = drive_service.files ().get_media (fileId = file_id)
  fh = io.BytesIO ()
  downloader = MediaIoBaseDownload (fh, req)
  done = False

  try:
    # Download Process
    with tqdm (total = 100, unit = "%", desc = f"Downloading {file_name} file") as pbar:
      while done is False:
        status, done = downloader.next_chunk ()
        pbar.update (int (status.progress () * 100) - pbar.n)

    # Writing Process in the Environment
    with open (os.path.join (local_path, file_name), "wb") as f:
      f.write (fh.getvalue())

    try:
      fh.seek (0)
      return fh.read().decode("utf-8")
    except UnicodeDecodeError:
      return None
  except HttpError:
    return None

In [ ]:
# Download Dataset Files from Google Drive.
if IS_COLAB:
  try:
    # Identify dataset folder ID
    print_mess ("Search datasets folder...", "yellow")
    dataset_folder_id = search_folder (main_folder_id = main_dataset_folder_id,
                                       initial_folder_name = common_dataset_folder_name)
    if dataset_folder_id is None:
      raise ValueError ("The folder was not found.")
    print_mess (f"The most recent dataset folder ID is: {dataset_folder_id}\n")

    # Search Evaluation Datasets
    print_mess ("Search evaluation dataset...", "yellow")
    dataset_id = search_dataset (dataset_folder_id = dataset_folder_id,
                                  dataset_name = dataset_name)
    if dataset_id is None:
      raise ValueError (f"The '{dataset_name}' was not found.")
    print_mess (f"The file '{dataset_name}' was found with ID: {dataset_id}")

    # Download and Save Dataset files
    print_mess ("\nDownload and Save Evaluation Dataset...", "yellow")
    dataset_file = download_and_save_file (file_id = dataset_id,
                                            file_name = dataset_name,
                                            local_path = dataset_download_dir)
    if dataset_file is None:
      raise ValueError (f"The '{dataset_name}' file could not be downloaded and saved.")

    # Confirmation Message
    print_mess ("\n\nDataset downloaded successfully.", "green")
  except Exception as e:
    print_mess ("Something went wrong.", "red")
    print_mess (f"Error: {e}", "yellow")

Search datasets folder...
The most recent dataset folder ID is: 1yH731dZ2LTKuorgDGINDxLkuewAmxK_C

Search evaluation dataset...
The file 'RMI_Intentions_Evaluation_Dataset.json' was found with ID: 148rQjLqrxOui9DezaxJ1-BhcukVS5rMX

Download and Save Evaluation Dataset...




Dataset downloaded successfully.


# Model Download

In [ ]:
# Download Model Files from Google Drive.
if IS_COLAB:
  try:
    # Identify model folder ID
    print_mess ("Search model folder...", "yellow")
    model_folder_id = search_folder (main_folder_id = main_model_folder_id,
                                       initial_folder_name = common_model_folder_name)
    if model_folder_id is None:
      raise ValueError ("The folder was not found.")
    print_mess (f"The most recent model folder ID is: {model_folder_id}\n")

    # Identify model files
    print_mess ("Search model files...", "yellow")
    search_result = search_files (folder_id = model_folder_id)
    if search_result is None:
      raise ValueError ("The files were not found.")
    id_model_file_list = search_result[0]
    name_model_file_list = search_result[1]
    for i in id_model_file_list:
      print_mess((f"The file '{name_model_file_list[id_model_file_list.index(i)]}'"
                  f" was found with ID: {i}"))

    # Download and Save Model Files
    print_mess ("\nDownload and Save Model Files...", "yellow")
    for i in id_model_file_list:
      download_and_save_file (file_id = i,
                              file_name = name_model_file_list[id_model_file_list.index(i)],
                              local_path = model_download_dir)

    # Confirmation Message
    print_mess ("\n\nModel downloaded successfully.", "green")
  except Exception as e:
    print_mess ("Something went wrong.", "red")
    print_mess (f"Error: {e}", "yellow")


Search model folder...
The most recent model folder ID is: 1fTHHVV1x6x1sbjJ-7AWttW6BbLUwyf7b

Search model files...
The file 'adapter_model.safetensors' was found with ID: 1441KbRvjTp93KAp2vplItBoPVLskfHEn
The file 'README.md' was found with ID: 1lGat2c808maxTJLpKZLUumCBgLAF-bXg
The file 'special_tokens_map.json' was found with ID: 1pBlSWvlvvXzhkUTW0SbhtRbS4-QNPP8N
The file 'tokenizer_config.json' was found with ID: 1OsW-HLvlodW-KCL90o70SEpWFOLWkEIO
The file 'sentencepiece.bpe.model' was found with ID: 1FjVF3KvDrRWx7wfxU84z-QTS70m_D7TS
The file 'adapter_config.json' was found with ID: 1meSUb4QPavg7-D1ESH6562O0q-54YDv6
The file 'tokenizer.json' was found with ID: 1nfBILXLrXda0SzSNaPt3L0wTeAzS13CT

Download and Save Model Files...




Model downloaded successfully.


# Load the Model

In [ ]:
# Max Sequence Length
max_source_length = 300
max_target_length = 1024 # 2048
max_length = max_target_length

In [ ]:
# QLoRA Configuration
if torch.cuda.is_available ():
  bnb_config = BitsAndBytesConfig (
      load_in_4bit = True,
      bnb_4bit_use_double_quant = True,
      bnb_4bit_quant_type = "nf4",
      bnb_4bit_compute_dtype = torch.bfloat16
  )
else:
  bnb_config = BitsAndBytesConfig (
      load_in_4bit = True,
      bnb_4bit_use_double_quant = True,
      bnb_4bit_quant_type = "nf4"
  )

In [ ]:
# Define the model name
model_name = "NLLB-200-Distilled-600M-Trained"

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained (
    model_name,
    quantization_config=bnb_config,
    device_map = "auto"
)
tokenizer = NllbTokenizerFast.from_pretrained (model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

# Processing Dataset

In [ ]:
# Processing Train and Test Datasets Function
def processing_dataset (raw_dataset:dict) -> Union[None, list]:
  processed_data_list = []

  try:
    # Process raw Data
    for conversation in raw_dataset["conversations"]:
      # Input = system_message + user_operator
      system_msg = conversation[0]["value"]
      user_msg = conversation[1]["value"]
      input_text = "Instruction: " + system_msg + " " + user_msg

      # Output = model_answer
      target_text = conversation[2]["value"]

      # Save Process Data
      processed_data_list.append({
          'input_text': input_text,
          'target_text': target_text
      })

    return processed_data_list
  except KeyError:
    return None

In [ ]:
# Dataset Tokenize Function
def tokenize_function (samples:datasets.formatting.formatting.LazyBatch) -> datasets.arrow_dataset.Dataset:

  # Tokenize Input Text
  model_inputs = tokenizer (
      samples["input_text"],
      max_length = max_source_length,
      truncation = True,
      padding = "max_length"
  )

  # Tokenize Target Text (Labels)
  labels = tokenizer (
      samples["target_text"],
      max_length = max_target_length,
      truncation = True,
      padding = "max_length"
  )

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
# Processing Datasets
try:
  print_mess ("Processing Evaluation Dataset...", "yellow")
  eval_dataset = Dataset.from_list (processing_dataset (json.loads (dataset_file)))
  if eval_dataset is None:
    raise ValueError ("Evaluation Dataset could not be processed.")
  print_mess ("Evaluation Dataset.")
  print (eval_dataset)
except Exception as e:
  print_mess ("Something went wrong.", "red")
  print_mess (f"Error: {e}", "yellow")

Processing Evaluation Dataset...
Evaluation Dataset.
Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 19
})


# Model Execution

In [ ]:
generated_outputs = []
generation_times = []

device = torch.device ("cuda" if torch.cuda.is_available () else "cpu")
model.to(device)

for item in tqdm(eval_dataset):
  # Generate Tensors
  inputs = tokenizer(
      item["input_text"],
      return_tensors="pt",
      max_length=max_source_length,
      truncation=True
  ).to(device)

  # Start time measurement
  start_time = datetime.datetime.now()

  # Generate Outputs
  generated_output = model.generate(
      input_ids=inputs["input_ids"],
      attention_mask=inputs["attention_mask"],
      max_length=max_target_length,
      use_cache=True
  )

  # End time measurement
  end_time = datetime.datetime.now()
  generation_times.append(end_time - start_time)

  # Decode output to add in list
  decoded_output = tokenizer.decode(generated_output[0], skip_special_tokens=True)
  generated_outputs.append(decoded_output)

print_mess("\nGeneration times:")
print(generation_times)


100%|██████████| 19/19 [19:48<00:00, 62.53s/it]


Generation times:
[datetime.timedelta(seconds=93, microseconds=633431), datetime.timedelta(seconds=67, microseconds=410526), datetime.timedelta(seconds=59, microseconds=756006), datetime.timedelta(seconds=59, microseconds=893821), datetime.timedelta(seconds=59, microseconds=699343), datetime.timedelta(seconds=59, microseconds=833649), datetime.timedelta(seconds=59, microseconds=947076), datetime.timedelta(seconds=59, microseconds=966906), datetime.timedelta(seconds=60, microseconds=253125), datetime.timedelta(seconds=61, microseconds=30340), datetime.timedelta(seconds=60, microseconds=394139), datetime.timedelta(seconds=60, microseconds=473753), datetime.timedelta(seconds=61, microseconds=479254), datetime.timedelta(seconds=60, microseconds=693526), datetime.timedelta(seconds=60, microseconds=802973), datetime.timedelta(seconds=61, microseconds=104927), datetime.timedelta(seconds=60, microseconds=713055), datetime.timedelta(seconds=60, microseconds=736244), datetime.timedelta(seconds=

In [ ]:
generated_outputs[0]

'``` @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> . @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> . @prefix xsd: <http://www.w3.org/2001/XMLSchema#> . @prefix t: <http://www.w3.org/2006/time#> . @prefix ex: <http://example.org/> . @prefix icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/> . @prefix imo: <http://tio.models.tmforum.org/tio/v3.6.0/IntentManagmentOntology/> . @prefix log: <http://tio.models.tmforum.org/tio/v3.6.0/LogicalOperators/> . @prefix quan: <http://tio.models.tmforum.org/tio/v3.6.0/QuantityOntology/> . @prefix set: <http://tio.models.tmforum.org/tio/v3.6.0/SetOperators/> . @prefix fun: <http://tio.models.tmforum.org/tio/v3.6.0/FunctionOntology/> . ex:VLAN200MemoryOverflowIntent a icm:Intent ; log:allOf (ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation ex:ConnectionPersistenceExpectation ex:ConnectionPersistenceExpectation) . ex:MemoryOverflowExpectation a icm:DeliveryExpectation ; icm:target

In [ ]:
generated_outputs[1]

'``` @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> . @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> . @prefix xsd: <http://www.w3.org/2001/XMLSchema#> . @prefix t: <http://www.w3.org/2006/time#> . @prefix ex: <http://example.org/> . @prefix icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/> . @prefix imo: <http://tio.models.tmforum.org/tio/v3.6.0/IntentManagmentOntology/> . @prefix log: <http://tio.models.tmforum.org/tio/v3.6.0/LogicalOperators/> . @prefix quan: <http://tio.models.tmforum.org/tio/v3.6.0/QuantityOntology/> . @prefix set: <http://tio.models.tmforum.org/tio/v3.6.0/SetOperators/> . @prefix fun: <http://tio.models.tmforum.org/tio/v3.6.0/FunctionOntology/> . ex:VLAN50MemoryOverflowIntent a icm:Intent ; log:allOf (ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation) . ex:MemoryOverflowExpectation a icm:DeliveryExpectation ; icm:target ex:VLAN50Target ; log:allOf (ex:MemoryOverf

In [ ]:
generated_outputs[2]

'``` @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> . @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> . @prefix xsd: <http://www.w3.org/2001/XMLSchema#> . @prefix t: <http://www.w3.org/2006/time#> . @prefix ex: <http://example.org/> . @prefix icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/> . @prefix imo: <http://tio.models.tmforum.org/tio/v3.6.0/IntentManagmentOntology/> . @prefix log: <http://tio.models.tmforum.org/tio/v3.6.0/LogicalOperators/> . @prefix quan: <http://tio.models.tmforum.org/tio/v3.6.0/QuantityOntology/> . @prefix set: <http://tio.models.tmforum.org/tio/v3.6.0/SetOperators/> . @prefix fun: <http://tio.models.tmforum.org/tio/v3.6.0/FunctionOntology/> . ex:VLAN300MemoryOverflowIntent a icm:Intent ; log:allOf (ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation) . ex:MemoryOverflowExpectation a icm:DeliveryExpectation ; icm:target ex:VLAN300SwitchesTarget ; log:allOf (ex:M

In [ ]:
generated_outputs[3]

'``` @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> . @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> . @prefix xsd: <http://www.w3.org/2001/XMLSchema#> . @prefix t: <http://www.w3.org/2006/time#> . @prefix ex: <http://example.org/> . @prefix icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/> . @prefix imo: <http://tio.models.tmforum.org/tio/v3.6.0/IntentManagmentOntology/> . @prefix log: <http://tio.models.tmforum.org/tio/v3.6.0/LogicalOperators/> . @prefix quan: <http://tio.models.tmforum.org/tio/v3.6.0/QuantityOntology/> . @prefix set: <http://tio.models.tmforum.org/tio/v3.6.0/SetOperators/> . @prefix fun: <http://tio.models.tmforum.org/tio/v3.6.0/FunctionOntology/> . ex:VLAN10MemoryOverflowIntent a icm:Intent ; log:allOf (ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation ex:MemoryOverflowExpectation ex:ConnectionPersistenceExpectation ex:ConnectionPersistenceExpectation) . ex:MemoryOverflowExpectation a icm:DeliveryExpectation ; icm:target 

In [ ]:
generated_outputs[10]

'``` @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> . @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> . @prefix xsd: <http://www.w3.org/2001/XMLSchema#> . @prefix t: <http://www.w3.org/2006/time#> . @prefix ex: <http://example.org/> . @prefix icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/> . @prefix imo: <http://tio.models.tmforum.org/tio/v3.6.0/IntentManagmentOntology/> . @prefix log: <http://tio.models.tmforum.org/tio/v3.6.0/LogicalOperators/> . @prefix quan: <http://tio.models.tmforum.org/tio/v3.6.0/QuantityOntology/> . @prefix set: <http://tio.models.tmforum.org/tio/v3.6.0/SetOperators/> . @prefix fun: <http://tio.models.tmforum.org/tio/v3.6.0/FunctionOntology/> . ex:MPLSTunnelIntent a icm:Intent ; log:allOf (ex:MPLSTunnelExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation) . ex:BackupLinkExpectation a icm:PropertyExpectation ; icm:target ex:BackupLinkExpectation ; icm:target ex:BackupL

In [ ]:
generated_outputs[11]

'``` @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> . @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> . @prefix xsd: <http://www.w3.org/2001/XMLSchema#> . @prefix t: <http://www.w3.org/2006/time#> . @prefix ex: <http://example.org/> . @prefix icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/> . @prefix imo: <http://tio.models.tmforum.org/tio/v3.6.0/IntentManagmentOntology/> . @prefix log: <http://tio.models.tmforum.org/tio/v3.6.0/LogicalOperators/> . @prefix quan: <http://tio.models.tmforum.org/tio/v3.6.0/QuantityOntology/> . @prefix set: <http://tio.models.tmforum.org/tio/v3.6.0/SetOperators/> . @prefix fun: <http://tio.models.tmforum.org/tio/v3.6.0/FunctionOntology/> . ex:Intent a icm:Intent ; log:allOf (ex:BackupLinkExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation) . ex:BackupLinkExpectation a icm:PropertyExpectation ; icm:target ex:BackupLinkTarget ; log:allOf (

In [ ]:
generated_outputs[12]

'``` @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> . @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> . @prefix xsd: <http://www.w3.org/2001/XMLSchema#> . @prefix t: <http://www.w3.org/2006/time#> . @prefix ex: <http://example.org/> . @prefix icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/> . @prefix imo: <http://tio.models.tmforum.org/tio/v3.6.0/IntentManagmentOntology/> . @prefix log: <http://tio.models.tmforum.org/tio/v3.6.0/LogicalOperators/> . @prefix quan: <http://tio.models.tmforum.org/tio/v3.6.0/QuantityOntology/> . @prefix set: <http://tio.models.tmforum.org/tio/v3.6.0/SetOperators/> . @prefix fun: <http://tio.models.tmforum.org/tio/v3.6.0/FunctionOntology/> . ex:MPLSBackupTunnelIntent a icm:Intent ; log:allOf (ex:MPLSBandwidthExpectation ex:BackupTunnelExpectation ex:BackupTunnelExpectation ex:BackupTunnelExpectation) . ex:BackupTunnelExpectation a icm:DeliveryExpectation ; icm:target ex:BackupTunnelTarget ; log:allOf (ex:BackupTunnelExpect

In [ ]:
generated_outputs[13]

'``` @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> . @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> . @prefix xsd: <http://www.w3.org/2001/XMLSchema#> . @prefix t: <http://www.w3.org/2006/time#> . @prefix ex: <http://example.org/> . @prefix icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/> . @prefix imo: <http://tio.models.tmforum.org/tio/v3.6.0/IntentManagmentOntology/> . @prefix log: <http://tio.models.tmforum.org/tio/v3.6.0/LogicalOperators/> . @prefix quan: <http://tio.models.tmforum.org/tio/v3.6.0/QuantityOntology/> . @prefix set: <http://tio.models.tmforum.org/tio/v3.6.0/SetOperators/> . @prefix fun: <http://tio.models.tmforum.org/tio/v3.6.0/FunctionOntology/> . ex:MPLSIntent a icm:Intent ; log:allOf (ex:MPLSTunnelExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation ex:BackupLinkExpectation) . ex:BackupLinkExpectation a icm:PropertyExpectation ; icm:target ex:BackupLinkExpectation ; icm:target ex:BackupLinkTar

# Processing results

In [ ]:
# Result processing
tmf_format_outputs = []
for output in generated_outputs:
  # Remove triple backticks from the output
  cleaned_output = output.replace("```", "").strip()
  tmf_format_outputs.append(cleaned_output)

print_mess("Results processed successfully", "green")

Results processed successfully


# Save Results

In [ ]:
# Get today's date
now = datetime.datetime.now()
today = now.strftime("%Y-%m-%d_%H-%M-%S")

# Define the folder ID where the new folder will be created
folder_id = "1o7uzwgct9frGLs65X0Dezhrlj7tRGfaq"

# Define the name of the new folder
new_folder_name = f"Model_Results_{today}"

In [ ]:
# Create new folder
try:
  file_metadata = {
      "name": new_folder_name,
      "parents": [folder_id],
      "mimeType": "application/vnd.google-apps.folder"
  }
  folder = drive_service.files().create(
      body = file_metadata,
      fields = "id"
  ).execute()
  created_folder_id = folder.get("id")
  print_mess(f"Folder '{new_folder_name}' created successfully with ID: {created_folder_id}", "green")
except HttpError as error:
  print_mess(f"An error occurred: {error}", "red")
  created_folder_id = None

Folder 'Model_Results_2025-10-25_21-24-31' created successfully with ID: 19zZ7bht3M-RLnQirr26JASYJ2SzT_nJ3


In [ ]:
# Define file name
save_file_name = "generated_tmf_format_outputs.json"

# Convert the list to a JSON string
json_generated_outputs = json.dumps(tmf_format_outputs, indent=2)

# Create file content as bytes
save_file_content = json_generated_outputs.encode("utf-8")

In [ ]:
# Define save file metadata
save_file_metadata = {
    "name": save_file_name,
    "parents": [created_folder_id],
    "mimeType": "application/json"
}

In [ ]:
# Create MediaIoBaseUpload object
media = MediaIoBaseUpload(
    io.BytesIO(save_file_content),
    mimetype = "application/json",
    resumable = True
)

In [ ]:
# Upload file
try:
  uploaded_file = drive_service.files().create(
      body = save_file_metadata,
      media_body = media,
      fields = "id"
  ).execute()
  print_mess(f"File '{save_file_name}' uploaded successfully with ID: {uploaded_file.get('id')}", "green")
except HttpError as error:
  print_mess(f"An error occurred: {error}", "red")

File 'generated_tmf_format_outputs.json' uploaded successfully with ID: 1NtDyw6xCa6t96fDYsI9DjPvsHMoYbSr1


In [ ]:
print_mess("All done!")

All done!
